This version of the MTOC Workflow uses label free segmentations to generate segmentations of tubulin MTOC.

In [14]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_3d_wrapper
from aicssegmentation.core.seg_dot import dot_2d
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, edge_preserving_smoothing_3d, image_smoothing_gaussian_3d
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects 
from skimage.filters import threshold_otsu
from skimage.morphology import ball, binary_closing, remove_small_objects, dilation, erosion, disk, square, binary_dilation
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import vesselness3D

import pandas as pd
from tifffile import imsave

In [47]:
# manifest_csv = pd.read_csv('manifest.csv')

In [ ]:
# tubulin_only = manifest_csv[(manifest_csv['structure_name'] == 'TUBA1B')]
# tubulin_only.head()

<h1> LABEL FREE TUBULIN NUCLEUS (NOT CURRENTLY IN USE) <h1>

In [70]:
# filename = "//allen/aics/assay-dev/users/Sandi/aics-segmentation/lookup_table_demo/tubulin/static-tubulin-segmentations/all-raw-imgs/20210312_N02_002-alignV2-Scene-09-P9-D02.tiff"
# reader = AICSImage(filename) 
# IMG = reader.data.astype(np.float32)

# print(IMG.shape)
# #view(single_fluorescent_view(IMG))

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 2, 50, 1200, 1800)


In [71]:
# N_CHANNELS = IMG.shape[2]
# MID_SLICE = np.int(0.5*IMG.shape[3])

# fig, ax = plt.subplots(1, N_CHANNELS, figsize=(18,16), dpi=72, facecolor='w', edgecolor='k')
# if N_CHANNELS==1:
#     ax.axis('off')
#     ax.imshow(IMG[0,0,0,MID_SLICE,:,:], cmap=plt.cm.gray)
# else:
#     for channel in range(N_CHANNELS):
#         ax[channel].axis('off')
#         ax[channel].imshow(IMG[0,0,channel,MID_SLICE,:,:], cmap=plt.cm.gray)

In [72]:
# #####################
# ## PARAMETER ##
# structure_channel = 0
# #####################

# struct_img0 = IMG[0,structure_channel,:,:,:].copy() 
# #view(single_fluorescent_view(struct_img0))

In [73]:
# from aicssegmentation.core.pre_processing_utils import suggest_normalization_param
# suggest_normalization_param(struct_img0)

mean intensity of the stack: 154.14483642578125
the standard deviation of intensity of the stack: 137.34506225585938
0.9999 percentile of the stack intensity is: 1990.0
minimum intensity of the stack: 8.0
maximum intensity of the stack: 5114.0
suggested upper range is 13.5, which is 2008.3031768798828
suggested lower range is 1.0, which is 16.799774169921875
So, suggested parameter for normalization is [1.0, 13.5]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


In [74]:
# # intensity normalization v2
# intensity_scaling_param = [8000] # original: 8000 # current: 6000
# struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param) #struct_img0
# #view(single_fluorescent_view(struct_img))

intensity norm: min-max norm with upper bound 8000


In [75]:
# # gaussian smoothing
# gaussian_smoothing_sigma = 1 # original: 1
# struct_img_smooth = image_smoothing_gaussian_slice_by_slice(struct_img, sigma=gaussian_smoothing_sigma)
# #view(single_fluorescent_view(struct_img_smooth))

In [76]:
# # applying mask object threshold
# bw, object_for_debug = MO(struct_img_smooth, global_thresh_method='tri', object_minArea=1200, return_object=True)
# #view(single_fluorescent_view(bw))

In [77]:
# # apply dilation to masked nucleus image

# from scipy import ndimage as ndi

# # make a little 3D diamond:
# diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
# # dilate 30x with it
# dilated_img = ndi.binary_dilation(bw, diamond, iterations=10) # original: 5

<h1> MTOC TUBULIN WORKFLOW <h1>

In [352]:
filename2 = "//allen/aics/assay-dev/users/Sandi/aics-segmentation/lookup_table_demo/tubulin/static-tubulin-segmentations/all-raw-imgs/20210312_N02_002-alignV2-Scene-02-P2-D02.tiff"
reader = AICSImage(filename2) 
IMG2 = reader.data.astype(np.float32)


The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


In [353]:
#####################
## PARAMETER ##
structure_channel = 0
#####################

struct_img0_2 = IMG2[0,structure_channel,:,:,:].copy() 
#view(single_fluorescent_view(struct_img0))

<h1> 1: PRE-PROCESSING <h1>

In [355]:
# intensity normalization v2
intensity_scaling_param2 = [8.0,10] # original: 8000 # current: 6000
struct_img2 = intensity_normalization(struct_img0_2, scaling_param=intensity_scaling_param2) #struct_img0

view(single_fluorescent_view(struct_img2))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [356]:
from aicssegmentation.core.pre_processing_utils import suggest_normalization_param
suggest_normalization_param(struct_img2)

mean intensity of the stack: 0.09529032558202744
the standard deviation of intensity of the stack: 0.08840659260749817
0.9999 percentile of the stack intensity is: 1.0
minimum intensity of the stack: 6.280619687520916e-12
maximum intensity of the stack: 1.0
suggested upper range is 10.0, which is 0.9793562516570091
suggested lower range is 1.0, which is 0.006883732974529266
So, suggested parameter for normalization is [1.0, 10.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


In [357]:
# gaussian smoothing
gaussian_smoothing_sigma2 = 1 # original: 1
structure_img_smooth2 = image_smoothing_gaussian_slice_by_slice(struct_img2, sigma=gaussian_smoothing_sigma2)

<h1> 2: CORE SEGMENTATION ALGORITHMS <h1>

In [358]:
# otsu thresholding: higher value, more segmentation
th = 3.2* threshold_otsu(structure_img_smooth2) # original at 1.4 # current: 3.2 
overall_shape = dilation(
    remove_small_objects(structure_img_smooth2 > th, min_size=10, connectivity=1, in_place=False), #min_size = 10
    ball(1)
)
fixed_shape = erosion(
    hole_filling(overall_shape, hole_min=0, hole_max=5, fill_2d=True), #, hole_min= 0 hole_max = 5
    ball(1)
)

In [359]:
# 2d spot filter
s2 = np.zeros_like(fixed_shape)
for z in range(structure_img_smooth2.shape[0]):
    zslice = dot_2d(structure_img_smooth2[z,], 2)
    s2[z, :, :] = zslice > 0.2 #original: 0.03 current  
fixed_shape = np.logical_or(s2, fixed_shape)

In [360]:
# 2d erosion for thinning
for z in range(fixed_shape.shape[0]):
    zslice= fixed_shape[z,]
    zslice = erosion(zslice, disk(1))
    fixed_shape[z, :, :] = zslice

response_vessel = vesselness3D(structure_img_smooth2, sigmas=[1], tau=1, whiteonblack=True)

seg = np.logical_or(response_vessel > 2.0, fixed_shape) # response_vessel > 0.28 increasing parameter thins out lines

seg = hole_filling(seg, hole_min=0, hole_max=15, fill_2d=True)# hole_min=0 # hole_max=15

In [361]:
# applying mask object threshold
bw2, object_for_debug = MO(seg, global_thresh_method='tri', object_minArea=1200, return_object=True)

Converting input from bool to <class 'numpy.uint8'> for compatibility.


In [362]:
thin_dist_preserve=0.09
thin_dist=2
bw_thin = topology_preserving_thinning(bw2>0, thin_dist_preserve, thin_dist)

<h1> 3: POST-PROCESSING <h1>

In [363]:
# Remove large area 
## PARAMETERS for this step ##
minArea = 1 # original 1
################################

#seg2 = remove_small_objects(bw_thin>0, min_size=minArea, connectivity=1, in_place=True)
seg2 = remove_small_objects(bw_thin>0, min_size=minArea, connectivity=1, in_place=True)

In [364]:
# save the file
out=seg2.astype(np.uint8)
out[out>0]=255
imsave("mtoc-d02s02.tiff", out)
#imsave("mtoc-seg-v9.tiff", seg2)

<h1> COMBINING BOTH SEGMENTED IMAGES (NOT CURRENTLY IN USE) <h1>

In [313]:
# seg_NUC = dilated_img
# seg_MTOC = seg2

In [179]:
# save the hybrid image


# seg_MTOC[seg_NUC==0] = 0
# imsave("mtoc-seg-combinedv7.tiff", seg2)

<h1> Extra Code (Not Currently in Use) <h1>

In [ ]:
# bw_binary = bw.astype(int)*1  

In [24]:
#img_norm = intensity_normalization(struct_img, scaling_param=[20, 25])# original: [2,20] #seg 12 & 14: [50,55] #seg 13: [30,35] seg14-2.0 [25 30] 14check3: [45 50] 14check4: 50 55
#img_smooth = edge_preserving_smoothing_3d(img_norm, numberOfIterations=30)

In [25]:
# #applying f3 filter and 3D wrapping

# # PARAMETERS for this step ##
# f3_param = [[1, 0.03]] # original: [[1, 0.01]]  v3: [[1.25, 0.02]]

# ###############################

# bw = filament_3d_wrapper(structure_img_smooth, f3_param) #structure_img_smooth
# bw = dot_3d_wrapper(structure_img_smooth, f3_param) #structure_img_smooth

In [ ]:
# # applying 3D Spot Filter
# s3_param = [[1, 0.009]] # v4: [[1, 0.05]] lower 2nd parameter, more seg
# ################################

# bw_extra = dot_3d_wrapper(structure_img_smooth, s3_param) # structure_img_smooth